In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
class BeeDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
queen_train = np.load('queen_train.npy')
nonqueen_train = np.load('nonqueen_train.npy')

queen_val = np.load('queen_val.npy')
nonqueen_val = np.load('nonqueen_val.npy')

queen_test = np.load('queen_test.npy')
nonqueen_test = np.load('nonqueen_test.npy')

queen_val.shape, nonqueen_val.shape

In [ ]:
x_train = np.vstack((queen_train, nonqueen_train))
ones = np.ones(7000)
zeros = np.zeros(7000)
y_train = np.concatenate((ones, zeros))
x_train.shape

In [ ]:
x_val = np.vstack((queen_val, nonqueen_val))
ones = np.ones(1000)
zeros = np.zeros(1000)
y_val = np.concatenate((ones, zeros))
x_val.shape

In [ ]:
x_test = np.vstack((queen_test, nonqueen_test))
ones = np.ones(2000)
zeros = np.zeros(2000)
y_test = np.concatenate((ones, zeros))
x_test.shape

In [ ]:
transform = transforms.Compose([
    # transforms.ToPILImage(),  # Convert numpy array to PIL Image
    # transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [ ]:
train_dataset = BeeDataset(x_train, y_train, transform=transform)
val_dataset = BeeDataset(x_val, y_val, transform=transform)
test_dataset = BeeDataset(x_test, y_test, transform=transform)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = models.resnet50(weights='IMAGENET1K_V1')

# Modify the first convolutional layer to accept 1 input channel instead of 3
model.conv1 = nn.Conv2d(1, 64, kernel_size=(5,5), stride=1, bias=False)

# Modify the final fully connected layer to output 2 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Move the model to the specified device (e.g., GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)
model


In [ ]:
# Training and validation loop
num_epochs = 100
train_accs = []
val_accs = []
train_losses = []
val_losses = []
best_val_loss = float('inf')  # Initialize with infinity
lr_str = np.format_float_positional(lr, trim='-')


for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_corrects = 0

    for train_images, train_labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
        train_images = train_images.to(device)
        train_labels = train_labels.long().to(device)

        optimizer.zero_grad()

        outputs = model(train_images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, train_labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * train_images.size(0)
        train_corrects += torch.sum(preds == train_labels.data)

    train_loss /= len(train_dataloader.dataset)
    train_acc = train_corrects.double() / len(train_dataloader.dataset)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validation
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_dataloader, desc=f"Epoch {epoch+1} Validation"):
            val_images = val_images.to(device)
            val_labels = val_labels.long().to(device)

            val_outputs = model(val_images)
            _, preds = torch.max(val_outputs, 1)
            loss = criterion(val_outputs, val_labels)

            val_loss += loss.item() * val_images.size(0)
            val_corrects += torch.sum(preds == val_labels.data)

    val_loss /= len(val_dataloader.dataset)
    val_acc = val_corrects.double() / len(val_dataloader.dataset)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    torch.save(model.state_dict(), f'/kaggle/working/curent_model_{lr_str}.pth')
    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'/kaggle/working/best_model_{lr_str}.pth')
        
print(f'Best Validation Loss: {best_val_loss:.4f}')

In [ ]:
# Load the best model
best_model = model
best_model.load_state_dict(torch.load(f'/kaggle/working/best_model_{lr_str}.pth'))

# Predict on the test set
best_model.eval()
test_predictions = []
true_labels = []
with torch.no_grad():
    for test_images, test_labels in tqdm(test_dataloader, desc="Test Set"):
        test_images = test_images.to(device)
        test_labels = test_labels.long().to(device)

        test_outputs = best_model(test_images)
        _, preds = torch.max(test_outputs, 1)
        test_predictions.extend(preds.cpu().numpy())
        true_labels.extend(test_labels.cpu().numpy())

# Evaluate the model on the test set
num_correct = sum([1 for i in range(len(test_predictions)) if test_predictions[i] == true_labels[i]])
test_accuracy = num_correct / len(test_predictions)

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Plotting train and validation losses
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Validation Losses over Epochs')
plt.legend()
plt.show()